In [45]:
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from datetime import datetime, timedelta
import pickle
from netCDF4 import Dataset
import nexradaws
import os
import metpy
from ungridded_section_spin import quality_control_spin

In [10]:
#Read in the sounding data from the spreadsheet
#Bring in the sounding data
CIsoundings=np.genfromtxt('SupercellSoundings/environment_cicases_Final.csv',skip_header=1,delimiter=',', dtype=str)
Oldsoundings=np.genfromtxt('SupercellSoundings/environment_Oldcases.csv',delimiter=',', dtype=str)
# ETsoundings=np.genfromtxt('PythonEnvVars1.csv', skip_header=1,delimiter=',')
# ETsoundings[ETsoundings==-9999]=np.nan

NewSoundings=np.genfromtxt('SupercellSoundingsNew/environment_cicases1022.csv', delimiter=',', dtype=str)

TORsondes = pickle.load(open('TORsondes.pkl', 'rb'))
NTORsondes = pickle.load(open('NTORsondes.pkl', 'rb'))
ETORsondes = pickle.load(open('ETORsondes.pkl', 'rb'))
ENTORsondes = pickle.load(open('ENTORsondes.pkl', 'rb'))
TTorsondes = pickle.load(open('TTorsondes.pkl', 'rb'))
TNTsondes = pickle.load(open('TNTsondes.pkl', 'rb'))
STsondes = pickle.load(open('STsondes.pkl', 'rb'))
SNsondes = pickle.load(open('SNsondes.pkl', 'rb'))

ALL_sites1 = pickle.load(open('ALLVARSite.pkl', 'rb'))
ALL_years1 = pickle.load(open('ALLVARYear.pkl', 'rb'))
ALL_months1 = pickle.load(open('ALLVARMonth.pkl', 'rb'))
ALL_days1 = pickle.load(open('ALLVARDay.pkl', 'rb'))
ALL_hours1 = pickle.load(open('ALLVARHour.pkl', 'rb'))
ALL_TN1 = pickle.load(open('ALLVARTor_Non.pkl', 'rb'))

ALL_b_t1 = np.zeros((ALL_TN1.shape))
ALL_b_t1[ALL_TN1=='T'] = 1.0

#Fix the incorrect entry here
ALL_sites1[44] = 'KTLX'
ALL_years1[44] = 2019
ALL_months1[44] = 3
ALL_days1[44] = 23
ALL_hours1[44] = 23
ALL_TN1[44] = 'N'

#Fix second incorrect entry here
ALL_sites1[54] = 'KFDR'
ALL_years1[54] = 2013
ALL_months1[54] = 4
ALL_days1[54] = 18
ALL_hours1[54] = 1
ALL_TN1[54] = 'T'

#Fix third incorrect entry here
ALL_sites1[61] = 'KENX'
ALL_years1[61] = 2014
ALL_months1[61] = 7
ALL_days1[61] = 3
ALL_hours1[61] = 21
ALL_TN1[61] = 'N'

obsd_TOR = pickle.load(open('sp6_dir_TOR.pkl', 'rb'))
obsd_NTOR = pickle.load(open('sp6_dir_NTOR.pkl', 'rb'))
obsd_ETOR = pickle.load(open('sp6_dir_ETOR.pkl', 'rb'))
obsd_ENTOR = pickle.load(open('sp6_dir_ENTOR.pkl', 'rb'))
obsd_TTor = pickle.load(open('sp6_dir_TTor.pkl', 'rb'))
obsd_TNT = pickle.load(open('sp6_dir_TNT.pkl', 'rb'))
obsd_ST = pickle.load(open('sp6_dir_ST.pkl', 'rb'))
obsd_SN = pickle.load(open('sp6_dir_SN.pkl', 'rb'))

obspd_TOR = pickle.load(open('sp6_sp_TOR.pkl', 'rb'))
obspd_NTOR = pickle.load(open('sp6_sp_NTOR.pkl', 'rb'))
obspd_ETOR = pickle.load(open('sp6_sp_ETOR.pkl', 'rb'))
obspd_ENTOR = pickle.load(open('sp6_sp_ENTOR.pkl', 'rb'))
obspd_TTor = pickle.load(open('sp6_sp_TTor.pkl', 'rb'))
obspd_TNT = pickle.load(open('sp6_sp_TNT.pkl', 'rb'))
obspd_ST = pickle.load(open('sp6_sp_ST.pkl', 'rb'))
obspd_SN = pickle.load(open('sp6_sp_SN.pkl', 'rb'))

#Create the full arrays for storm speed and direction
#all_dir = np.concatenate([obsd_TOR, obsd_NTOR, obsd_ETOR, obsd_ENTOR, obsd_TTor, obsd_TNT, obsd_ST, obsd_SN], axis=0)
#all_spd = np.concatenate([obspd_TOR, obspd_NTOR, obspd_ETOR, obspd_ENTOR, obspd_TTor, obspd_TNT, obspd_ST, obspd_SN], axis=0)


NewCases = np.genfromtxt('SPORK_rerun_new.csv', delimiter=',', usecols=(0,2,3,4,7,17),skip_header=1, dtype=int)
NewCases1 = np.genfromtxt('SPORK_rerun_new.csv', delimiter=',', usecols=(0,2,3,4,7,17), dtype=str, skip_header=1)

print(NewCases[0,:])
print(NewCases1[0,:])

ALL_sites2 = NewCases1[:,0]
ALL_years2 = NewCases[:,1]
ALL_months2 = NewCases[:,2]
ALL_days2 = NewCases[:,3]
ALL_hours2 = NewCases[:,4]
ALL_TN2 = np.copy(ALL_sites2)
ALL_TN2[:] = 'TR'
ffds2 = NewCases[:,5]
print(ffds2)


NewCases2 = np.genfromtxt('New2023CSV.csv', delimiter=',', skip_header=1, dtype=int)
NewCases3 = np.genfromtxt('New2023CSV.csv', delimiter=',', dtype=str, skip_header=1)


ALL_sites3 = NewCases3[:,0]
ALL_years3 = NewCases2[:,2]
ALL_months3 = NewCases2[:,3]
ALL_days3 = NewCases2[:,4]
ALL_hours3 = NewCases2[:,7]
ALL_TN3 = np.copy(ALL_sites3)
ALL_TN3[:] = 'TR'
ffds3 = NewCases2[:,14]
print(ffds3)
#ALL_TN = pickle.load(open('ALLVARTor_Non.pkl', 'rb'))


ALL_sites = np.concatenate([ALL_sites1, ALL_sites2, ALL_sites3], axis=0)
ALL_years = np.concatenate([ALL_years1, ALL_years2, ALL_years3], axis=0)
ALL_months = np.concatenate([ALL_months1, ALL_months2, ALL_months3], axis=0)
ALL_days = np.concatenate([ALL_days1, ALL_days2, ALL_days3], axis=0)
ALL_hours = np.concatenate([ALL_hours1, ALL_hours2, ALL_hours3], axis=0)
ALL_TN = np.concatenate([ALL_TN1, ALL_TN2, ALL_TN3], axis=0)

[  -1 2021    6   19    0  220]
['KIND' '2021' '6' '19' '0' '220']
[220 160 190 145 200 140 160 180 110 170 120 170 165 140 150 140 140 170
 170 150 150 130 150 130 140 190 180 180 180 180 130  70 170 200 200 190
 190 120 130 140 200 200 180 220 190 180 190 160 160 150 160 150 160 180
 140 150 140 170 150 160 180 180 210 230 190 230 180 180 150 130 190 240
 170 130 190 160 130 150 180 180 190 190 180 230 150 150 200 230 170 190
 140 150 150 220 160 190 180 180 160 170 180 150 160 150]
[180 180 180 150 180 190 140 180 130 150 160 190 150 160 160 160 170 190
 180 180 170 140 210 160 160 220 210 210 160 180 190 170 150 180 170 180
 190 150 170 180 160 160 160 180 160 160 210 150 180 140 120 140 170 160
 170 150 220 170 160 190 210 210 200 200 240 210 200 220 130 210 190 210
 230 180 210 180 150 190 180 170 160 140 140 200 200 190 210 240 190 240
 200 160 200 190 170 190 200 190 210]


In [5]:
"""
Function for extracting the radar column above a target
given position in latitude, longitude

"""

import numpy as np
import pandas as pd
import xarray as xr

from pyart.core.transforms import antenna_vectors_to_cartesian


def column_vertical_profile(
    radar, latitude, longitude, azimuth_spread=3, spatial_spread=3
):
    """
    Given the location (in latitude, longitude) of a target, return the rays
    that correspond to radar column above the target, allowing for user
    defined range of azimuths and range gates to be included within this
    extraction.

    Parameters
    ----------
    radar : pyart.core.Radar Object
        Py-ART Radar Object from which distance to the target, along
        with gates above the target, will be calculated.
    latitude : float, [degrees]
        Latitude, in degrees North, of the target.
    longitude : float, [degrees]
        Longitude, in degrees East, of the target.
    azimuth_spread : int
        Number of azimuth angles to include within extraction list
    spatial_range : int
        Number of range gates to include within the extraction

    Function Calls
    --------------
    sphere_distance
    for_azimuth
    get_sweep_rays
    subset_fields
    assemble_column

    Returns
    -------
    column : xarray
        Xarray Dataset containing the radar column above the target for
        the various fields within the radar object.

    References
    ----------
    Murphy, A. M., A. Ryzhkov, and P. Zhang, 2020: Columnar Vertical
    Profile (CVP) Methodology for Validating Polarimetric Radar Retrievals
    in Ice Using In Situ Aircraft Measurements. J. Atmos. Oceanic Technol.,
    37, 1623–1642, https://doi.org/10.1175/JTECH-D-20-0011.1.

    Bukovčić, P., A. Ryzhkov, and D. Zrnić, 2020: Polarimetric Relations for
    Snow Estimation—Radar Verification. J. Appl. Meteor. Climatol.,
    59, 991–1009, https://doi.org/10.1175/JAMC-D-19-0140.1.
    """

    # Define the spatial range to use within extraction
    spatial_range = radar.range["meters_between_gates"] * spatial_spread

    # Define a dictionary structure to contain the extracted features
    total_moment = {key: [] for key in radar.fields.keys()}
    total_moment.update({"height": [], "time_offset": []})

    # Define the start of the radar volume
    base_time = pd.to_datetime(radar.time["units"][14:]).to_numpy()

    # call the sphere_distance function
    dis = sphere_distance(
        radar.latitude["data"][0], latitude, radar.longitude["data"][0], longitude
    )
    # calculate forward azimuth angle
    forazi = for_azimuth(
        radar.latitude["data"][0], latitude, radar.longitude["data"][0], longitude
    )

    # Iterate through radar sweeps, extract desired section
    for sweep in radar.iter_slice():
        moment = {key: [] for key in radar.fields.keys()}
        zgates = []
        gate_time = []

        # call the new sweep rays
        center, spread = get_sweep_rays(
            radar.azimuth["data"][sweep], forazi, azimuth_spread=azimuth_spread
        )

        # add the start indice of each ray
        center = [x + sweep.start for x in center]
        spread = [x + sweep.start for x in spread]
        # Correct the spread indices to remove the centerline
        spread = [x for x in spread if x not in center]

        # For the ray(s) directly over the target, extract and average fields
        for ray in center:
            # Convert gates from antenna or cartesian coordinates
            (rhi_x, rhi_y, rhi_z) = antenna_vectors_to_cartesian(
                radar.range["data"],
                radar.azimuth["data"][ray],
                radar.elevation["data"][ray],
                edges=False,
            )
            # Calculate distance to target
            rhidis = np.sqrt((rhi_x**2) + (rhi_y**2)) * np.sign(rhi_z)
            # Calculate target gate
            tar_gate = np.nonzero(np.abs(rhidis[0, :] - dis) < spatial_range)[
                0
            ].tolist()
            # Subset the radar fields for the target locations
            subset = subset_fields(radar, ray, tar_gate)
            # Add back to the total dictionary
            moment = {key: moment[key] + subset[key] for key in moment}
            # Add radar elevation to height gates
            # to define height as center of each gate above sea level
            zgates.append(np.ma.mean(rhi_z[0, tar_gate] + radar.altitude["data"][0]))
            # Determine the time for the individual gates
            gate_time.append(radar.time["data"][ray])

        # Convert to Cartesian Coordinates
        # Determine the center of each gate for the subsetted rays.
        for ray in spread:
            (rhi_x, rhi_y, rhi_z) = antenna_vectors_to_cartesian(
                radar.range["data"],
                radar.azimuth["data"][ray],
                radar.elevation["data"][ray],
                edges=False,
            )
            # Calculate distance to target
            rhidis = np.sqrt((rhi_x**2) + (rhi_y**2)) * np.sign(rhi_z)
            # Calculate target gate
            tar_gate = np.nonzero(np.abs(rhidis[0, :] - dis) < spatial_range)[
                0
            ].tolist()
            # Subset the radar fields for the target locations
            subset = subset_fields(radar, ray, tar_gate)
            # Add back to the sweep dictionary
            moment = {key: moment[key] + subset[key] for key in moment}
            # Add radar elevation to height gates
            # to define height as center of each gate above sea level
            zgates.append(np.ma.mean(rhi_z[0, tar_gate] + radar.altitude["data"][0]))
            # Determine the time for the individual gates
            gate_time.append(radar.time["data"][ray])

        # Average all azimuth moments into a single value for the sweep
        for key in total_moment:
            if key == "height":
                total_moment[key].append(np.ma.mean(np.ma.masked_invalid(zgates)))
            elif key == "time_offset":
                total_moment[key].append(np.round(np.ma.mean(np.array(gate_time)), 4))
            else:
                total_moment[key].append(
                    np.round(np.ma.mean(np.ma.masked_invalid(moment[key])), 4)
                )

    # Add the base time for the radar
    total_moment.update({"base_time": base_time})

    # Convert to xarray
    return assemble_column(radar, total_moment, forazi, dis, latitude, longitude)



def sphere_distance(radar_latitude, target_latitude, radar_longitude, target_longitude):
    """
    Calculated of the great circle distance between radar and target

    Assumptions
    -----------
    Radius of the Earth = 6371 km / 6371000 meters
    Distance is calculated for a smooth sphere
    Radar and Target are at the same altitude (need to check)

    Parameters
    ----------
    radar_latitude : float, [degrees]
        latitude of the radar in degrees
    target_latitude : float, [degrees]
        latitude of the target in degrees
    radar_longitude : float, [degrees]
        longitude of the radar in degrees
    target_longitude : float, [degrees]
        longitude of the target in degress

    Returns
    -------
    distance : float, [meters]
        Great-Circle Distance between radar and target in meters
    """
    # check if latitude, longitudes are valid
    check_latitude(radar_latitude)
    check_latitude(target_latitude)
    check_longitude(radar_longitude)
    check_longitude(target_longitude)

    # convert latitudeitude/longitudegitudes to radians
    radar_latitude = radar_latitude * (np.pi / 180.0)
    target_latitude = target_latitude * (np.pi / 180.0)
    radar_longitude = radar_longitude * (np.pi / 180.0)
    target_longitude = target_longitude * (np.pi / 180.0)

    # difference in latitudeitude
    d_latitude = target_latitude - radar_latitude
    # difference in longitudegitude
    d_longitude = target_longitude - radar_longitude

    # Haversine formula
    numerator = (np.sin(d_latitude / 2.0) ** 2.0) + np.cos(radar_latitude) * np.cos(
        target_latitude
    ) * (np.sin(d_longitude / 2.0) ** 2.0)
    distance = 2 * 6371000 * np.arcsin(np.sqrt(numerator))

    # return the output
    return distance



def for_azimuth(radar_latitude, target_latitude, radar_longitude, target_longitude):
    """
    Calculation of inital bearing alongitudeg a great-circle arc
    Known as Forward Azimuth Angle.

    Assumptions
    -----------
    Radius of the Earth = 6371 km / 6371000 meters
    Distance is calculatitudeed for a smooth sphere
    Radar and Target are at the same altitude (need to check)

    Parameters
    ----------
    radar_latitude : float, [degrees]
        latitude of the radar in degrees
    target_latitude : float, [degrees]
        latitude of the target in degrees
    radar_longitude : float, [degrees]
        longitude of the radar in degrees
    target_longitude : float, [degrees]
        longitude of the target in degress

    Returns
    -------
    azimuth : float, [degrees]
        azimuth angle from the radar where
        target is located within the scan.
        output is in degrees.
    """
    # check the input latitude/longitude
    check_latitude(radar_latitude)
    check_latitude(target_latitude)
    check_longitude(radar_longitude)
    check_longitude(target_longitude)

    # convert latitudeitude/longitudegitudes to radians
    radar_latitude = radar_latitude * (np.pi / 180.0)
    target_latitude = target_latitude * (np.pi / 180.0)
    radar_longitude = radar_longitude * (np.pi / 180.0)
    target_longitude = target_longitude * (np.pi / 180.0)

    # Differnce in longitudegitudes
    d_longitude = target_longitude - radar_longitude

    # Determine x,y coordinates for arc tangent function
    corr_y = np.sin(d_longitude) * np.cos(target_latitude)
    corr_x = (np.cos(radar_latitude) * np.sin(target_latitude)) - (
        np.sin(radar_latitude) * (np.cos(target_latitude) * np.cos(d_longitude))
    )

    # Determine forward azimuth angle
    azimuth = np.arctan2(corr_y, corr_x) * (180.0 / np.pi)

    # Return the output as a function of 0-360 degrees
    if azimuth < 0:
        azimuth += 360.0

    return azimuth



def get_field_location(radar, latitude, longitude):
    """
    Given the location (in latitude, longitude) of a target, extract the
    radar column above that point for further analysis.

    Parameters
    ----------
    radar : pyart.core.Radar Object
        Py-ART Radar Object from which distance to the target, along
        with gates above the target, will be calculated.
    latitude : float, [degrees]
        Latitude, in degrees North, of the target.
    longitude : float, [degrees]
        Longitude, in degrees East, of the target.

    Function Calls
    --------------
    sphere_distance
    for_azimuth
    get_column_rays

    Returns
    -------
    column : xarray DataSet
        Xarray Dataset containing the radar column above the target for
        the various fields within the radar object.
    """

    # Make sure latitude, longitudes are valid
    check_latitude(latitude)
    check_longitude(longitude)

    # initiate a dictionary to hold the gates above the radar.
    zgate = []

    # initiate a diciontary to hold the moment data.
    moment = {key: [] for key in radar.fields.keys()}

    # call the sphere_distance function
    dis = sphere_distance(
        radar.latitude["data"][0], latitude, radar.longitude["data"][0], longitude
    )

    # call the for_azimuth function
    azim = for_azimuth(
        radar.latitude["data"][0], latitude, radar.longitude["data"][0], longitude
    )

    # call the get_column_ray function
    ray = get_column_rays(radar, azim)

    # Determine the center of each gate for the subsetted rays.
    (rhi_x, rhi_y, rhi_z) = antenna_vectors_to_cartesian(
        radar.range["data"],
        radar.azimuth["data"][ray],
        radar.elevation["data"][ray],
        edges=False,
    )
    # Calculate distance from the x,y coordinates to target
    rhidis = np.sqrt((rhi_x**2) + (rhi_y**2)) * np.sign(rhi_z)
    for i in range(len(ray)):
        tar_gate = np.argmin(abs(rhidis[i, 1:] - (dis)))
        for key in moment:
            if radar.fields[key]["data"][ray[i], tar_gate] is np.ma.masked:
                moment[key].append(np.nan)
            else:
                moment[key].append(radar.fields[key]["data"][ray[i], tar_gate])
        # Add radar elevation to height gates
        # to define height as center of each gate above sea level
        zgate.append(rhi_z[i, tar_gate] + radar.altitude["data"][0])

    # Determine the time at the center of each ray within the column
    # Define the start of the radar volume as a numpy datetime object for xr
    # We take advantage of the "seconds since " portion of the units string
    base_time = pd.to_datetime(radar.time["units"][14:]).to_numpy()
    # Convert Py-ART radar object time (time since volume start) to time delta
    # Add to base time to have sequential time within the xr Dataset
    # for easier future merging/work
    combined_time = []
    for i in range(len(ray)):
        delta = pd.to_timedelta(radar.time["data"][ray[i]], unit="s")
        total_time = base_time + delta
        combined_time.append(total_time.to_numpy())

    # Create a blank list to hold the xarray DataArrays
    ds_container = []
    da_meta = [
        "units",
        "standard_name",
        "long_name",
        "valid_max",
        "valid_min",
        "coordinates",
    ]
    # Convert the moment dictionary to xarray DataArray.
    # Apply radar object meta data to DataArray attribute
    for key in moment:
        if key != "height":
            da = xr.DataArray(
                moment[key], coords=dict(height=zgate), name=key, dims=["height"]
            )
            for tag in da_meta:
                if tag in radar.fields[key]:
                    da.attrs[tag] = radar.fields[key][tag]
            # Append to ds container
            ds_container.append(da.to_dataset(name=key))

    # Add additional DataArrays 'base_time' and 'time_offset'
    # if not present within the radar object.
    da_base = xr.DataArray(base_time, name="base_time")
    da_offset = xr.DataArray(
        combined_time, coords=dict(height=zgate), name="time_offset", dims=["height"]
    )
    ds_container.append(da_base.to_dataset(name="base_time"))
    ds_container.append(da_offset.to_dataset(name="time_offset"))

    # Create a xarray DataSet from the DataArrays
    column = xr.merge(ds_container)

    # Assign Attributes for the Height and Times
    height_des = (
        "Height Above Sea Level [in meters] for the Center of Each"
        + " Radar Gate Above the Target Location"
    )
    column.height.attrs.update(
        long_name="Height of Radar Beam",
        units="m",
        standard_name="height",
        description=height_des,
    )

    column.base_time.attrs.update(long_name="UTC Reference Time", units="seconds")

    time_long = "Time in Seconds Since Volume Start"
    time_des = (
        "Time in Seconds Since Volume Start that Cooresponds"
        + " to the Center of Each Height Gate"
        + " Above the Target Location"
    )
    column.time_offset.attrs.update(
        long_name=time_long, units="seconds", description=time_des
    )

    # Assign Global Attributes to the DataSet
    column.attrs["distance_from_radar"] = str(np.around(dis / 1000.0, 3)) + " km"
    column.attrs["azimuth"] = str(np.around(azim, 3)) + " degrees"
    column.attrs["latitude_of_location"] = str(latitude) + " degrees"
    column.attrs["longitude_of_location"] = str(longitude) + " degrees"
    return column



def get_column_rays(radar, azimuth):
    """
    Given the location (in latitude,longitude) of a target, return the rays
    that correspond to radar column above the target.

    Parameters
    ----------
    radar : Radar Object
        Py-ART Radar Object from which distance to the target, along
        with gates above the target, will be calculated.
    azimuth : float,int
        forward azimuth angle from radar to target in degrees.

    Returns
    -------
    nrays : List
        radar ray indices that correspond to the column above a
        target location.
    """
    if isinstance(azimuth, int) or isinstance(azimuth, float) is True:
        if (azimuth <= 0) or (azimuth >= 360):
            raise ValueError("azimuth not valid (not between 0-360 degrees)")
    else:
        raise TypeError(
            "radar azimuth type not valid."
            " Please convert input to be an int or float."
        )
    # define a list to hold the valid rays
    rays = []
    # check to see which radar scan
    if radar.scan_type == "rhi":
        for i in range(radar.sweep_number["data"].shape[0]):
            nstart = radar.sweep_start_ray_index["data"][i]
            nstop = radar.sweep_end_ray_index["data"][i]
            counter = 0
            for j in range(nstart, nstop):
                if abs(radar.azimuth["data"][nstart + counter] - azimuth) < 1:
                    rays.append(nstart + counter)
                counter += 1
    else:
        # taken from pyart.graph.RadarDisplay.get_azimuth_rhi_data_x_y_z
        for sweep in radar.iter_slice():
            sweep_azi = radar.azimuth["data"][sweep]
            nray = np.argmin(np.abs(sweep_azi - azimuth))
            rays.append(nray + sweep.start)
    # make sure rays were found
    if len(rays) == 0:
        raise ValueError("No rays were found between azimuth and target")

    return rays



def get_sweep_rays(sweep_azi, azimuth, azimuth_spread=0):
    """
    Extract the specific rays for a given azimuth from a radar sweep

    Azimuth spread determines the +/- degrees azimuth to include within
    the extraction by multipling the azimuth resolution by input value.

    Parameters
    ----------
    radar_sweep : pyart.core.radar object
        Radar Sweep from which the rays are extracted from
    azimuth : float [degrees]
        Forward Azimuth Angle from Radar to Target in Degreees
    Azimuth_Spread : int
        Number of azimuth angles to include within extraction list

    Returns
    -------
    center_rays : list [integers]
        List of integers cooresponding to ray indices within the azimuth
        directly over the target
    spread_rays : list [integers]
        List of integers cooresponding to ray indices within the spread
        of azimuths emcompassing the target

    """
    # determine resolution of azimuth angles
    resolution = np.round((sweep_azi[1] - sweep_azi[0]), 3)

    centerline = np.nonzero(np.abs(sweep_azi - azimuth) < 0.5)[0].tolist()
    spread = np.nonzero(np.abs(sweep_azi - azimuth) < (resolution * azimuth_spread))[
        0
    ].tolist()

    return centerline, spread


def subset_fields(radar, ray, target_gates):
    """
    Parameter
    ---------
    radar : pyart.core.radar object
        Radar Sweep from which fields are extracted from the target locations
    target_gates : list
        List containing indices for the gates of interest

    Returns
    -------
    fields : dict
        dictionary containing averaged subset fields for target location

    """
    # initiate a diciontary to hold the moment data.
    moment = {key: [] for key in radar.fields.keys()}

    # Iterate over input rays and average according to input method
    # future - allow users to input weights for spatial averaging
    for key in moment:
        if key != "height":
            if np.ma.all(radar.fields[key]["data"][ray, target_gates]) is np.ma.masked:
                moment[key].append(np.nan)
            else:
                moment[key].append(
                    np.ma.mean(radar.fields[key]["data"][ray, target_gates])
                )

    return moment


def assemble_column(radar, total_moment, azimuth, distance, latitude, longitude):
    """
    With a dictionary containing the extracted fields from a radar sweep,
    assemble individual gates and fields into an xarray DataSet

    Parameters
    ----------
    total_moment : dict
        Dictionary containing the extracted fields from the radar object.
        File requires at least the height of the individual gates and
        the start time of the volumetric scan.
    azimuth : float, [degrees]
        azimuth angle from the radar where
        target is located within the scan.
        output is in degrees.
    distance : float, [meters]
        Great-Circle Distance between radar and target in meters
    latitude : float, [degrees]
        Latitude of the target in degrees
    longitude : float, [degrees]
        Longitude of the target in degrees

    Returns
    -------
    column : xarray DataSet
        Xarray Dataset containing the radar column above the target for
        the various fields within the radar object.

    """
    # Create a blank list to hold the xarray DataArrays
    ds_container = []
    da_meta = [
        "units",
        "standard_name",
        "long_name",
        "valid_max",
        "valid_min",
        "coordinates",
    ]
    # Skip these fields and apply meta data after creation
    # of the Xarray DataSet
    skip = ["height", "base_time"]
    # Convert the moment dictionary to xarray DataArray.
    # Apply radar object meta data to DataArray attribute
    for key in total_moment:
        if key not in skip:
            # Convert Masked Array elements to NaNs for Xarray
            total_moment[key] = [
                np.nan if x is np.ma.masked else x for x in total_moment[key]
            ]
            # Convert to Xarray DataArray, set derived height as
            # dimension/coordinates
            da = xr.DataArray(
                total_moment[key],
                coords=dict(height=total_moment["height"]),
                name=key,
                dims=["height"],
            )
            # Add meta data for the radar fields
            if key != "time_offset":
                for tag in da_meta:
                    if tag in radar.fields[key]:
                        da.attrs[tag] = radar.fields[key][tag]
            # Append to ds container
            ds_container.append(da.to_dataset(name=key))

    # Create a xarray DataSet from the DataArrays
    column = xr.merge(ds_container)

    # Add the scan times back into the merged column
    column["base_time"] = total_moment["base_time"]
    column.base_time.attrs.update(
        long_name=(
            "Start time of individual radar scan volumes "
            + " from which column are extracted "
        ),
        units="UTC Time",
    )

    # Assign Attributes for the Height and Times
    height_des = (
        "Height Above Sea Level [in meters] for the Center of Each"
        + " Radar Gate Above the Target Location"
    )
    column.height.attrs.update(
        long_name="Height of Radar Beam",
        units="m",
        standard_name="height",
        description=height_des,
    )

    time_long = "Time in Seconds Since Volume Start to the Center of Each Gate"
    time_des = (
        "Time in Seconds Since Volume Start (i.e. base_time) that Cooresponds"
        + " to the Center of Each Height Gate"
        + " Above the Target Location"
    )
    column.time_offset.attrs.update(
        long_name=time_long, units="seconds", description=time_des
    )

    # Add latitude, longitude as variable in extracted column
    column["latitude"] = latitude
    column.latitude.attrs.update(
        long_name="Latitude of Location Column is Extracted Above", units="deg"
    )
    column["longitude"] = longitude
    column.longitude.attrs.update(
        long_name="Longitude of Location Column is Extracted Above", units="deg"
    )

    # Assign Global Attributes to the DataSet
    column.attrs["distance_from_radar"] = str(np.around(distance / 1000.0, 3)) + " km"
    column.attrs["azimuth"] = str(np.around(azimuth, 3)) + " degrees"
    column.attrs["latitude_of_location"] = str(latitude) + " degrees"
    column.attrs["longitude_of_location"] = str(longitude) + " degrees"

    # Drop duplicated heights, keep latest value
    column = column.drop_duplicates(dim="height", keep="last")

    return column


def check_latitude(latitude):
    """
    Function to check if input latitude is valid for type and value.

    Parameters
    ----------
    latitude : int, float
        Latitude of a location that should be between 90S and 90N

    """
    if (
        isinstance(latitude, int)
        or isinstance(latitude, float)
        or isinstance(latitude, np.floating)
    ) is True:
        if (latitude <= -90) or (latitude >= 90):
            raise ValueError(
                "Latitude not between -90 and 90 degrees, need to "
                "convert to values between -90 and 90"
            )
    else:
        raise TypeError(
            "Latitude type not valid, need to convert input to be an int or float"
        )


def check_longitude(longitude):
    """
    Function to check if input latitude is valid for type and value.

    Parameters
    ----------
    longitude : int, float
        Longitude of a location taht should be between 180W and 180E
    """
    if (
        isinstance(longitude, int)
        or isinstance(longitude, float)
        or isinstance(longitude, np.floating)
    ) is True:
        if (longitude <= -180) or (longitude >= 180):
            raise ValueError(
                "Longitude not valid between -180 and 180"
                " degrees, need to convert to values between"
                "  -180 and 180"
            )
    else:
        raise TypeError(
            "Longitude type not valid, need to convert input to be an int or float"
        )

In [6]:
z_arr = np.arange(0,10.25,0.25)
print(z_arr.shape)
z_am = z_arr*1000

(41,)


In [34]:

def get_grid_time_offsets(dt, end_dt, station, cenlat_arr, cenlon_arr):

    n=0
    
    station=station
    start_dt=dt
    end_dt=end_dt
    calibration=0.0
    
    #Here, set the initial time of the archived radar loop you want.
    #Our specified time
    station = station
    #end_dt = dt + timedelta(hours=duration)
    
    #Set up nexrad interface
    conn = nexradaws.NexradAwsInterface()
    scans = conn.get_avail_scans_in_range(start_dt,end_dt,station)
    results = conn.download(scans, 'RadarFolder')
    
    #Actual algorithm code starts here
    #Create a list for the lists of arc outlines
    zdr_out_list = []
    heighttime_list = []
    for i,scan in enumerate(results.iter_success(),start=1):
    #Local file option:
        #Loop over all files in the dataset and pull out each 0.5 degree tilt for analysis
        try:
            radar1 = scan.open_pyart()
        except:
            print('bad radar file')
            continue
        #Local file option
        print('File Reading')
    
        #Calling ungridded_section; Pulling apart radar sweeps and creating ungridded data arrays
        [radar,radar_v,n,range_2d,last_height,rlons_h,rlats_h,ungrid_lons,ungrid_lats] = quality_control_spin(radar1,n,calibration)
    
        column = column_vertical_profile(radar_v, cenlat_arr[i], cenlon_arr[i], azimuth_spread=1, spatial_spread=1)
        print('time offsets', np.asarray(column['time_offset']), np.asarray(column['height']))
    
        time_interp = metpy.interpolate.interpolate_1d(z_am, np.asarray(column['height']), np.asarray(column['time_offset']))
        time_interp[np.isnan(time_interp)]=np.nanmin(time_interp)
        time_interp1 = time_interp-np.min(time_interp)
        heighttime_list.append(time_interp1)

        return heighttime_list

In [40]:
#Generate the file names for the postage stamps
j=1
#for i in range(len(ALL_years)):
#for i in np.arange(201,207,1):
for i in [0]:
    #try:
    print('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
    ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')

    ref = ncfile.variables['REFL'][:]
    kdp = ncfile.variables['KDP'][:]
    zdr = ncfile.variables['ZDR'][:]
    cc = ncfile.variables['CC'][:]
    zdr[ref < 20] = np.nan
    zdr[cc < 0.9] = np.nan
    kdp[ref < 20] = np.nan
    kdp[cc < 0.9] = np.nan
    nzdr = ncfile.variables['NZDR'][:]
    nzdr[ref[:,4,:,:] < 20] = 0.0
    nzdr[cc[:,4,:,:] < 0.95] = 0.0
    zdrd = ncfile.variables['ZDRD'][:]
    rot = ncfile.variables['ROT'][:]
    times = ncfile.variables['Times'][:]
    azim = ncfile.variables['azim'][:]
    vel = ncfile.variables['VEL'][:]
    lon_st = ncfile.variables['Lons'][:]
    lat_st = ncfile.variables['Lats'][:]

    ts1_arr = []
    for j in range(len(times)):

        ts2 = datetime.fromtimestamp(times[j])
        ts1_arr.append(ts2)

    heighttimes = get_grid_time_offsets(ts1_arr[0], ts1_arr[-1], str(ALL_sites[i]), lat_st[:,70,70], lon_st[:,70,70])
    # except:
    #     try:
    #         ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
    #     except:
    #         print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc is missing!')
    #         print(j)
    #         j=j+1
    

D:/NewSPORK_stamps/SPORK_RERUN201727KDGX.nc
Downloaded KDGX20170207_205357_V06
Downloaded KDGX20170207_204830_V06
Downloaded KDGX20170207_205925_V06
Downloaded KDGX20170207_204303_V06
Downloaded KDGX20170207_203748_V06
Downloaded KDGX20170207_203241_V06
Downloaded KDGX20170207_210453_V06
Downloaded KDGX20170207_211021_V06
Downloaded KDGX20170207_211548_V06
Downloaded KDGX20170207_212116_V06
Downloaded KDGX20170207_212643_V06
Downloaded KDGX20170207_213157_V06
Downloaded KDGX20170207_213717_V06
Downloaded KDGX20170207_214244_V06
Downloaded KDGX20170207_214812_V06
Downloaded KDGX20170207_215332_V06
Downloaded KDGX20170207_215852_V06
Downloaded KDGX20170207_220413_V06
Downloaded KDGX20170207_220933_V06
Downloaded KDGX20170207_221501_V06
Downloaded KDGX20170207_222014_V06
Downloaded KDGX20170207_222528_V06
Downloaded KDGX20170207_223034_V06
Downloaded KDGX20170207_223527_V06
Downloaded KDGX20170207_224032_V06
Downloaded KDGX20170207_224552_V06
Downloaded KDGX20170207_225111_V06
Downloaded 

AttributeError: module 'metpy' has no attribute 'interpolate'

In [23]:
print(ts1_arr[0])

2017-02-07 20:32:41


In [38]:
print(lat_st[i])

[[32.30880306 32.30881648 32.30882967 ... 32.30855795 32.30854073
  32.30852328]
 [32.31330564 32.31331906 32.31333225 ... 32.3130605  32.31304328
  32.31302583]
 [32.31780821 32.31782163 32.31783483 ... 32.31756305 32.31754583
  32.31752838]
 ...
 [32.92565597 32.92566955 32.92568292 ... 32.92540776 32.92539032
  32.92537266]
 [32.93015854 32.93017213 32.93018549 ... 32.92991031 32.92989287
  32.92987521]
 [32.93466112 32.93467471 32.93468807 ... 32.93441287 32.93439543
  32.93437776]]


In [28]:
print(lon_st[0,70,70])

-89.93365596046326


In [43]:
print(time_interp)
time_interp[np.isnan(time_interp)]=np.nanmin(time_interp)
print(time_interp-np.nanmin(time_interp))

print(time_interp[20]-np.nanmin(time_interp))


[         nan          nan          nan          nan 106.84530018
 150.24925    170.10574268 201.78834477 231.51454363 237.64419162
 243.96121671 250.63875096 256.2505172  260.86660462 265.48269205
 272.3758565  280.12841366 287.88097082 297.73371706 310.71062518
 323.6875333  336.66444142 346.5766375  348.91218437 351.24773125
 353.58327812 355.918825   358.18466537 360.1052926  362.02591982
 363.94654704 365.86717426 367.78780149 369.70842871 371.15239494
 372.58762401 374.02285308 375.45808215 376.89331122 378.32854028
 379.76376935]
[  0.           0.           0.           0.           0.
  43.40394982  63.2604425   94.94304459 124.66924345 130.79889144
 137.11591653 143.79345078 149.40521702 154.02130444 158.63739186
 165.53055632 173.28311348 181.03567064 190.88841688 203.865325
 216.84223312 229.81914124 239.73133732 242.06688419 244.40243107
 246.73797794 249.07352482 251.33936519 253.25999241 255.18061964
 257.10124686 259.02187408 260.9425013  262.86312853 264.30709476
 265.

In [10]:
column = column_vertical_profile(radar, 36.27, -87.06, azimuth_spread=1, spatial_spread=1)


In [17]:
print(np.asarray(column['height']))

[  665.5  1351.5  1729.5  2243.5  2791.5  3476.5   734.   1043.   4275.
  5351.   6639.   8238.5 10196.  12861.5 16187.5]


In [31]:
print(radar.longitude['data'][:][0])

-86.5625
